In [2]:
import numpy as np
import cv2
import time

import tkinter as tk
from PIL import Image, ImageTk

from matplotlib import pyplot as plt

In [3]:
def calculate_disparity_matrix(imgL, imgR,min_disp, max_disp, disp_size, window_radius):
    num_disp = max_disp-min_disp
    h, w = imgL.shape
    
    croph = [(h-disp_size)//2,(h+disp_size)//2]
    cropw = [(w-disp_size)//2,(w+disp_size)//2]

    kernel = np.ones([window_radius, window_radius])

    ref_img_cropped = imgL[croph[0]:croph[1],cropw[0]:cropw[1]]
    C = np.zeros(
        [disp_size, disp_size, num_disp])

    for d in range(0, num_disp):
        # shift image
        translation_matrix = np.float32([[1, 0, d], [0, 1, 0]])
        shifted_image = cv2.warpAffine(
            imgR, translation_matrix,
            (imgR.shape[1], imgR.shape[0]))
        shifted_image_cropped = shifted_image[croph[0]:croph[1],cropw[0]:cropw[1]]
        # calculate squared differences
        SAD = abs(np.float32(ref_img_cropped) - np.float32(shifted_image_cropped))
        # convolve with kernel and find SAD at each point
        C[:, :, d] = cv2.filter2D(SAD, -1, kernel)
        

    disparity = np.argmin(C, axis=2)
    return disparity

In [4]:
def get_distance_from_disparity(main_disp, f, b):
    distance = (f*b)/(main_disp*1000)
    return distance

In [37]:
#Parameters

#Stereo cameras
focal_length = 567.2 #pixels
baseline = 92.226 #mm

#Stereo matching algorithm
min_disp=0
max_disp=128
num_disp = max_disp-min_disp
disparity_area_size = 100
window_radius = 8

In [24]:
vidL = cv2.VideoCapture('../data/robotL.avi')
vidR = cv2.VideoCapture('../data/robotR.avi')
if (vidL.isOpened()==False or vidR.isOpened()==False):
  print("Error opening video")

# Read until video is completed
while(vidL.isOpened()):
      # Capture frame-by-frame
      retL, frameL = vidL.read()
      retR, frameR = vidR.read()
      if retL == True and retR == True:
        frameL_greyscale=cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
        frameR_greyscale=cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
        
    
        start = time.time()

        disparity = calculate_disparity_matrix(frameL_greyscale, frameR_greyscale,min_disp, max_disp, disparity_area_size, window_radius)
        main_disp = np.mean(disparity)
        
        distance = get_distance_from_disparity(main_disp, focal_length, baseline)
        print("FPS: ", 1/(time.time()-start))
        print("Distance to obstacle(m): ", distance)
		    
        # Scale disparity values for visualization 
        disparity_visual = np.uint8(disparity * 255 / num_disp)
          
        # Show frames
        cv2.imshow('Image', frameL)
        cv2.imshow('Disparity', disparity_visual)
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

      # Break the loop
      else: 
          break

# When everything done, release the video capture object
vidL.release()
vidR.release()
# Closes all the frames
cv2.destroyAllWindows()


FPS:  2.402785057982895
Distance to obstacle(m):  2.3246670221843004
FPS:  2.2708713905020135
Distance to obstacle(m):  2.31682953760875
FPS:  2.5926133799813327
Distance to obstacle(m):  2.3081040108366278
FPS:  2.4152069850508715
Distance to obstacle(m):  2.329598342148586
FPS:  2.626836507249581
Distance to obstacle(m):  2.310185698982929
FPS:  2.5367337052074266
Distance to obstacle(m):  2.333553166685171
FPS:  2.372660604017278
Distance to obstacle(m):  2.3342091736132566
FPS:  2.569538533545547
Distance to obstacle(m):  2.321976110236772
FPS:  2.628551875668606
Distance to obstacle(m):  2.324310092138984
FPS:  2.5442135771330965
Distance to obstacle(m):  2.3156740179068125
FPS:  2.7617161959557026
Distance to obstacle(m):  2.3259602329935927
FPS:  2.377371602047543
Distance to obstacle(m):  2.3284139316513657
FPS:  2.370808497890224
Distance to obstacle(m):  2.312980009816016
FPS:  2.5981234657223085
Distance to obstacle(m):  2.3114662482288235
FPS:  2.575869646123026
Distance to

In [42]:
class MainWindow():
    def __init__(self, window):
        self.window = window
        self.vidL = cv2.VideoCapture('../data/robotL.avi')
        self.vidR = cv2.VideoCapture('../data/robotR.avi')
        if (self.vidL.isOpened()==False or self.vidR.isOpened()==False):
            print("Error opening video")

        self.width = 1500
        self.height = 800
        self.interval = 1 # Interval in ms to get the latest frame

        self.distance_widget = tk.Label(root, text="Distance: 0", width=100)
        self.distance_widget.config(font=("Courier", 20))
        #self.distance_widget.grid()
        # Create canvas for gui
        self.canvas = tk.Canvas(self.window, width=self.width, height=self.height)
        self.canvas.grid(row=0, column=0)
        # Update gui
        self.update_gui()
    def update_gui(self):

        # Capture frame-by-frame
        retL, frameL = self.vidL.read()
        retR, frameR = self.vidR.read()
        if retL == True and retR == True:
            frameL_greyscale=cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
            frameR_greyscale=cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
            
        
            start = time.time()

            disparity = calculate_disparity_matrix(frameL_greyscale, frameR_greyscale,min_disp, max_disp, disparity_area_size, window_radius)
            main_disp = np.mean(disparity)
            
            distance = get_distance_from_disparity(main_disp, focal_length, baseline)
            print("FPS: ", 1/(time.time()-start))
            print("Distance to obstacle(m): ", distance)
                
            # Scale disparity values for visualization 
            disparity_visual = np.uint8(disparity * 255 / num_disp)
            disp_heatmap = cv2.applyColorMap(disparity_visual, cv2.COLORMAP_JET)
            # Display the resulting frame
            #print(frameL.shape)
            #img = cv2.imshow('Image', frameL)
            cv2.imshow('Disparity', disparity_visual)
            #Rearrange colors
            blue,green,red = cv2.split(frameL)
            self.image_rgb = cv2.merge((red,green,blue))
            self.image_rgb = Image.fromarray(self.image_rgb)
            self.image_rgb = ImageTk.PhotoImage(image=self.image_rgb, master=root)

           # Update image
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_rgb)

            blue,green,red = cv2.split(disp_heatmap)
            self.image_disp = cv2.merge((red,green,blue))
            self.image_disp = Image.fromarray(self.image_disp)
            self.image_disp = ImageTk.PhotoImage(image=self.image_disp, master=root)
            
            self.canvas.create_image(800, 0, anchor=tk.NW, image=self.image_disp)
            if(distance<0.8):
                self.distance_widget["text"] = "ALARM! Distance to obstacle: "+str(round(distance,2)) + "m"
                self.distance_widget.config(fg="red")
            else:
                self.distance_widget["text"] = "Distance to obstacle: "+str(round(distance,2))+ "m"
                self.distance_widget.config(fg="black")
            self.distance_widget.grid()
            # Repeat every 'interval' ms
            self.window.after(self.interval, self.update_gui)
            

        # Break the loop
        else:             
            # When everything done, release the video capture object
            vidL.release()
            vidR.release()
            # Closes all the frames
            cv2.destroyAllWindows()
            quit()
if __name__ == "__main__":
    root = tk.Tk()
    MainWindow(root)
    root.mainloop()

FPS:  6.113905137844429
Distance to obstacle(m):  2.6475514953360904
FPS:  8.407659757009384
Distance to obstacle(m):  2.6329728048320122
FPS:  8.71864353227057
Distance to obstacle(m):  2.616039487700102
FPS:  8.891040710553371
Distance to obstacle(m):  2.656411533500574
FPS:  8.780572768380505
Distance to obstacle(m):  2.603669644768081
FPS:  8.922171712766886
Distance to obstacle(m):  2.651160963347389
FPS:  8.490906440798744
Distance to obstacle(m):  2.653500958719273
FPS:  8.691975322713342
Distance to obstacle(m):  2.6438983897217145
FPS:  8.224512526079659
Distance to obstacle(m):  2.6250583217077996
FPS:  9.248455394982294
Distance to obstacle(m):  2.571441987130645
FPS:  8.818955384870932
Distance to obstacle(m):  2.641415229246617
FPS:  9.007320870601626
Distance to obstacle(m):  2.6570996845650194
FPS:  8.5984796923304
Distance to obstacle(m):  2.632416310632709
FPS:  8.94861214823665
Distance to obstacle(m):  2.608590509245407
FPS:  8.776842867396413
Distance to obstacle(m)